           A high-speed drone of size 1m (L) x 0.5m (W) is flying towards you at a speed of 50-75 km/hr. The drone is flying from a distance of 2500m. Altitude of drone at starting point is 1000m and your height is 1.5m. This drone has to be detected and tracked in real-time. The distance of drone from you to be calculated along with the speed of the drone. Please share a solution for the above problem statement. Your solution may use one or more cameras as you find it suitable.


1. Problem statement as you understood


 drone with dimensions 1m x 0.5m is flying towards a person (you) at a speed of 50-75 km/hr.
- The drone is initially at a distance of 2500m and an altitude of 1000m.
- The person (you) is at a height of 1.5m.
- The goal is to detect and track the drone in real-time, calculating its distance and speed.


2. Solution approach including hardware, software and tools required, assumptions, Challenges / limitations etc


In [ ]:
Hardware Requirements:
   * requirements include a modern CPU with at least 4 cores and a reasonable amount of RAM (8GB or more).
   * Having a GPU can significantly accelerate the inference speed.
   * External camera or webcam connected to the computer for real-time object detection

Software Requirements:
   * Windows or Linux distributions are supported by OpenCV and YOLO.
   * Python programming language (3.6 or higher).Integrated Development Environment (IDE) such as PyCharm, VS Code, or Jupyter Notebook.
   * OpenCV (cv2): For image and video processing, and deep neural networks integration.  

    
Approach:
   * Camera at a height of 1.5m (at the person's height) to capture the drone's movement(speed and distance) and detect its presence.
   * Install Python and necessary packages (opencv-python, numpy).
   * Download yolov3.weights and yolov3.cfg and place them in the working directory (https://pjreddie.com/darknet/yolo/).
   * Use OpenCV's cv2.dnn.readNet function to load the YOLOv3 model with yolov3.weights and yolov3.cfg.
                                
Limitations:
   * Without GPU acceleration, real-time performance may be limited, especially for high-resolution video streams.
   * Object detection accuracy depends on the quality of pre-trained weights (yolov3.weights) and configuration (yolov3.cfg)

                              

3.Sample code

In [ ]:
import cv2
import numpy as np
import time

#  paths to the YOLO files
yolo_weights_path = "C:/Users/HP/Downloads/yolov3.weights"
yolo_cfg_path = "C:/Users/HP/Downloads/yolov3 (2).cfg"

# Load YOLO
net = cv2.dnn.readNet(yolo_weights_path, yolo_cfg_path)
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Initialize the video capture
cap = cv2.VideoCapture(0)  # Replace with actual camera index or video file

# Parameters for distance calculation
focal_length = 800  # Assume a focal length (in pixels)
drone_real_width = 1  # Width of the drone in meters

# Initialize variables for speed calculation
prev_time = None
prev_distance = None

def calculate_distance(width_in_pixels):
    # Distance calculation using similar triangles
    distance = (drone_real_width * focal_length) / width_in_pixels
    return distance

while True:
    ret, frame = cap.read()
    if not ret:
        break

    height, width, channels = frame.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Information to show on the screen
    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Confidence threshold
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(class_ids[i])
            color = (0, 255, 0)
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Calculate distance
            distance = calculate_distance(w)
            cv2.putText(frame, f'Distance: {distance:.2f} m', (x, y - 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

            # Calculate speed
            current_time = time.time()
            if prev_time is not None and prev_distance is not None:
                time_diff = current_time - prev_time
                if time_diff > 0:
                    distance_diff = prev_distance - distance
                    speed = distance_diff / time_diff * 3.6  # Convert m/s to km/hr
                    cv2.putText(frame, f'Speed: {speed:.2f} km/hr', (x, y + h + 20),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

            # Update previous values
            prev_time = current_time
            prev_distance = distance

    cv2.imshow('Drone Tracker', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
